In [ ]:
# ##################################################
# The first step is to create a topic coverage matrix
# over the time slices
# used to run a granger test on topics
# ##################################################

# create a topic coverage matrix preset to 0

#don't worry about stats for the first 7 iterations
topiccoverage = []
sigtopics = []
i = 0
while i < len(docs_per_timeslice) :
    y = 0
    thistopic = []
    while y < num_topics:
        thistopic.append(float(0.0))
        y += 1
    topiccoverage.append(thistopic.copy())
    i += 1

# get the topic coverage per timeslice per doc from the model
timeslice = 0
for timeslicedocs in docs_per_timeslice :
    # for each doc in this timeslice
    for doc in timeslicedocs :
        # get the probability matrix
        probs = model.get_document_topics(bow[doc]).copy()
        #its a sparse array, prob[0] is the topic and prob[1] is the probabiltiy
        for prob in probs :
            topiccoverage[timeslice][prob[0]] += prob[1]
    timeslice += 1

# ##################################################
# Run the granger test on the topic coverage created above
# NOTE: In this algorithm, the granger test is not used to split topics
#       it is used to gather statistics, and to determine when the algorithm
#       has reached its peak confidence
#
# INPUT:
# timeslicetokencounts - we have the word coverage for each model token
# topiccoverage - and now we have the topic coverage
# topics - the current topic word probabilities
#
# OUTPUT:
# collected statistics, and update topic probabilities
# updated array newtopics = []
#
# ##################################################   
newtopics = []
#purity = float(0.0)
#purity_count = 0
lowtopics = []

causality_confidence = float(0.0)
causality_count = 0
sig_causality_confidence = float(0.0)
sig_causality_count = 0
#savelowpvalue = float(2.0) $$$
for ii in range(0, num_topics) :
    # ##################################################   
    #
    # run the granger test on the topic coverage
    # save significant topics
    #
    # ##################################################   
    tempgc = grangercausalitytests([[bets[i],topiccoverage[i][ii]] for i in range(0, len(bets))], the_lag, verbose=False)

    # ##################################################   
    #
    # check the lag - for now we just grab the smallest p-value in the lag
    #
    # ##################################################   
    lowpvalue = float(2.0)
    lowlag = 0
    for yy in range(1,the_lag+1) :
        stats = tempgc.get(yy)[0].get('ssr_ftest')
        if (stats[1] < lowpvalue) :
            lowlag = yy
            lowpvalue = stats[1]
    if lowpvalue < low_threshold : # lob_threshold is one of the system params, default 95% or .05 p-value - pretty standard
        sigtopics.append(ii)

    # grab the causality_confidence for every topic
    causality_confidence += lowpvalue
    causality_count += 1
    lowtopics.append(ii)
    
    # ##################################################   
    #
    # adjust the probabilities 
    #
    # ##################################################   
    wordprobstats = []
    tot = np.longdouble(0)
    count = 0

    # we adjust the probability based upon the prerun granger tests on the word streams
    for wordprob in topics[ii] :
        #pgranger is the pre-run word stream granger tests
        sig = float(wordprob * pgranger[count])
        tot += wordprob #keep the total for normalization
        
        # save the word list for normalization
        wordlist = [abs(sig), count]
        wordprobstats.append(wordlist.copy())
        count += 1
    for wordprobstat in wordprobstats :
        wordprobstat[0] = wordprobstat[0]/tot # normalize

    # create new topics, start with zero prob
    createtopic = zeroprobs.copy()
    for wordprobstat in wordprobstats :
        # add in the new probabilitys (already normalized)
        createtopic[wordprobstat[1]] = wordprobstat[0]
    newtopics.append(createtopic.copy())
    
    # ##################################################   
    #
    # just grab the significant_causality_confidence for statistics purpose
    #
    # ##################################################   
    if lowpvalue < low_threshold : # lob_threshold is one of the system params, default 95% or .05 p-value - pretty standard
        # grab the causality_conficence for sig topics
        sig_causality_confidence += lowpvalue
        sig_causality_count += 1
            

# ################################################## 
#
# if this is the largest causality_confidence
# save the topics for the output of the engine
#
# ##################################################   
if iteration >= itstoskip :
    if causality_confidence/causality_count < mostsigconf :
        mostsigconf = causality_confidence/causality_count
        mostsigtopics = sigtopics.copy()
        mostsigtopicwords = []
        for ii in range (0, len(sigtopics)) :
            mostsigtopicwords.append(model.get_topic_terms(sigtopics[ii], 10).copy())

        file_name = runname + str(iteration) 
        vis_name = save_path + file_name + ".sav"
        print ("\nSignificant Topics: ", sigtopics)


causality_confidence = 1-causality_confidence/causality_count 
print("causality_confidence = ", causality_confidence)
run_confidence.append(causality_confidence.copy())
sig_causality_confidence = 1-sig_causality_confidence/sig_causality_count 



# ##################################################
#
# using the new topics with updated probabilies
# return to the model
#
# ##################################################
